# Тарасова Анастасия

# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [26]:
### Если не работает загрузка on-line
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';', index_col = "Показатель")

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14 entries, pH to Окисляемость перманганатная
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Единица измерений  14 non-null     object
 1   Результат анализа  14 non-null     object
 2   Норматив           14 non-null     object
dtypes: object(3)
memory usage: 448.0+ bytes


In [46]:
df.loc['Цветность','Результат анализа'] = 0

In [47]:
df['Результат анализа'] = df['Результат анализа'].astype('float')

In [55]:
def norm(x,y):
    """ x результат
        y норматив"""
    if y.__contains__('в пределах'):
        list_val = re.findall('[0-9,]+',y)
        if x>=float(list_val[0]) and x<=float(list_val[1]):
            return 'норма'
        elif x< float(list_val[0]):
            return 'ниже нормы'
        else:
            'выше нормы'
    elif y.__contains__('не более'):
        val = re.findall('[0-9,]+',y)[-1].replace(',','.')
        if x > float(val):
            return 'выше нормы'
        else:
            return 'норма'

In [57]:
df['оценка'] = list(map(norm, df['Результат анализа'],df['Норматив']))

In [58]:
df

,Единица измерений,Результат анализа,Норматив,оценка
Показатель,,,,
pH,единицы pH,8.400,в пределах 6-9,норма
Запах,баллы,1.000,не более 2-3,норма
Цветность,градусы,0.000,не более 30,норма
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,норма
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",норма
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",норма
Нитраты (по NO3),мг/дм3,24.000,не более 45,норма
Фосфаты (P),мг/дм3,0.360,"не более 3,5",норма
Хлориды (Cl),мг/дм3,200.000,не более 350,норма


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [135]:
#аналитический расчет
import math 
from random import shuffle
import scipy.stats as st

round((math.factorial(5)/(math.factorial(3)*math.factorial(2)))/(math.factorial(9)/(math.factorial(3)*math.factorial(6))),3)

0.119

In [153]:
def exp(array, positiv_ivent, n):
    list_p = []
    for j in range(n):
        arr = array
        s=0
        for i in range(n):
            shuffle(arr)
            if positiv_ivent == arr[:len(positiv_ivent)]:
                s+=1
        list_p.append(s/n)
    return list_p

def interval(list_p):
    result = st.t.interval(alpha=0.99, df=len(list_p)-1,
              loc=np.mean(list_p),
              scale=st.sem(list_p))
    return result

In [155]:
# Расчитаем вероятность появления события ['orange','orange','orange'] в 1000 экспериментах 
# 1000 раз и посмотрим на среднее значение вероятности 

arr = ['orange','orange','orange','orange','orange',
          'apple','apple','apple','apple']
iv = ['orange','orange','orange']
p = exp(arr, iv, 1000)

In [148]:
pd.Series(p).describe()

count    1000.000000
mean        0.118658
std         0.010133
min         0.088000
25%         0.111000
50%         0.119000
75%         0.125000
max         0.153000
dtype: float64

In [154]:
# Доверительный интервал для среднего распределения вероятностей
interval(p)

(0.11783103781129102, 0.11948496218870898)

***Вывод:***

Вероятность события ['orange','orange','orange'] по результатам эксперимента надодится в интервале от 0.088 до 0.153 со средним 0.118, значение рассчитанное аналитически находится в границах доверительного интервала  (0.11783103781129102,0.11948496218870898). Исходя из этого мы можем говорить, что аналитический расчет подтверждается эксперементально.

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [159]:
# аналитическое решение
3/10 * 7/9

0.23333333333333334

In [156]:
arr = ['standard','standard','standard','standard','standard','standard','standard',
       'non-standard','non-standard','non-standard']
iv = ['non-standard','standard']

p = exp(arr, iv, 1000)

In [157]:
pd.Series(p).describe()

count    1000.000000
mean        0.234149
std         0.013257
min         0.195000
25%         0.225000
50%         0.234000
75%         0.243000
max         0.278000
dtype: float64

In [158]:
# Доверительный интервал для среднего распределения вероятностей
interval(p)

(0.23306710893156088, 0.2352308910684391)

***Вывод:***

Вероятность события ['non-standard','standard'] по результатам эксперимента надодится в интервале от 0.195 до 0.278 со средним 0.23414, значение рассчитанное аналитически находится в границах доверительного интервала  (0.23306710893156088, 0.2352308910684391). Исходя из этого мы можем говорить, что аналитический расчет подтверждается эксперементально.